# 제3 고지 : 고차 미분 계산
## STEP 31 : 고차 미분(구현 편)

이번 단계에서는 고차미분을 할 수 있도록 **역전파 계산시 계산그래프를 만들수 있도록** DeZero 를 변경하겠다.


### 32.1 새로운 DeZero로!

역전파 계산 그래프 연결을 위한 가장 중요한 변화는 `Variable` 의 `grad` 이다. 현재까지 `grad`는 `ndarray` 인스턴스를 참조했지만, 이제는 `Variable` 인스턴스를 참조할 수 있도록 아래와 같이 변경한다.

```python
class Variable:
    ...
    def backward(self,retain_grad=False):
        if self.grad is None:
            # self.grad = np.ones_like(self.data) 
            self.grad = Variable(np.ones_like(self.data)) 
        ...
            
```


### 32.2 함수 클래스의 역전파 

이제 DeZero의 구체적인 함수들을 수정하면 된다.  
지금까지 `dezero/core_simple.py` 파일에서 다음 DeZero의 함수들을 구현했다.

- `Add`
- `Mul`
- `Neg`
- `Sub`
- `Div`
- `Pow`

이번 단계에서는 해당 클래스들의 `backward()` 를 수정한 뒤 `dezero/core.py` 로 옮기겠다.

```python
class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y

    def backward(self, gy):
        return gy, gy
```

1. `Add` 의 경우 **역전파가 하는 일이 출력쪽에서 전해지는 미분값을 입력쪽으로 전달하는 역할**만 하기 때문에 수정할 것이 없다.


```python
class Mul(Function):
    def forward(self, x0, x1):
        y = x0 * x1
        return y

    def backward(self, gy):
        # x0, x1 = self.inputs[0].data, self.inputs[1].data
        x0,x1 = self.inputs 
        return gy * x1, gy * x0
```

2. `Mul`의 경우 수정 전에는 `self.inputs[0].data` 와 같이 `Variable` 안에 있는 `data(ndarray 인스턴스)`를 꺼내야했다.  이제는 `Variable` 인스턴스를 그대로 사용한다.  
여기서 주목해야 할것은 `gy*x0`/`gy*x1` 코드인데, 수정 후에는  `gy`,`x0`,`x1` 이 이제는 `Variable` 인스턴스이고, `Variable` 클래스의 `*` 연산자는 이미 오버로드 되어 있으므로 실행함과 동시에 `Mul` 클래스의 순전파가 호출되고, 이때 `Function.__call__()` 이 호출되어 계산 그래프가 만들어 진다.


나머지 `Sub`,`Neg`,`Div`,`Pow` 함수도 위와 동일하게 적용하면 다음과 같다.

```python
class Neg(Function):
    def forward(self, x):
        return -x

    def backward(self, gy):
        return -gy


class Sub(Function):
    def forward(self, x0, x1):
        y = x0 - x1
        return y

    def backward(self, gy):
        return gy, -gy


class Div(Function):
    def forward(self, x0, x1):
        y = x0 / x1
        return y

    def backward(self, gy):
        # x0, x1 = self.inputs[0].data, self.inputs[1].data
        x0, x1 = self.inputs
        gx0 = gy / x1
        gx1 = gy * (-x0 / x1**2)
        return gx0, gx1

class Pow(Function):
    def __init__(self, c):
        self.c = c

    def forward(self, x):
        y = x**self.c
        return y

    def backward(self, gy):
        (x,) = self.inputs
        c = self.c

        gx = c * x ** (c - 1) * gy
        return gx

```



### 32.3 역전파를 더 효율적으로 (모드 추가)

<18단계 메모리 절약 모드> 에서 역전파의 활성/비활성 모드를 도입했다. 이와 동일한 전략으로 **역전파를 1회만 전파시 역전파 비활성모드** 를 추가한다.  
이를 위해 `Variable` 의 `backward()` 에 다음을 추가한다.

```python
class Variable:
    ...

    def backward(self, retain_grad=False,create_graph=False): ## create_graph 추가로 모드 변경
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set = set()

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)

        add_func(self.creator)

        while funcs:
            f = funcs.pop()
            gys = [output().grad for output in f.outputs]  # output is weakref
            
            ###################################################
            with using_config("enable_backprop",create_graph):
                gxs = f.backward(*gys) # 메인 backward
            ###################################################
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    ############################
                    x.grad = x.grad + gx # 이 계산도 대상
                    ############################

                if x.creator is not None:
                    add_func(x.creator)

            if not retain_grad:
                for y in f.outputs:
                    y().grad = None  # y is weakref


```
구체적으로 `Mul` 클래스를 예시로 살펴보면,

1. 실제 역전파 처리를 `with using_config()` 에서 수행한다.
2. `create_graph`의 값에 따라 `Config.enable_backprop` 이 결정되고, `backward()` 계산을 1회 수행한다.
3. 역전파 계산시 `gy * x1` 은 `*` 오버로드된 연산자로 인해 `Mul()(gy,x1)`이 호출된다.
4. 이때 `Function.__call__()` 에서 `Config.enable_backprop` 이 참조됨에 따라 **계산 그래프의 연결 여부가 결정**된다.

여기서 `create_graph=False`를 기본 설정한 이유는 실무에서 역전파가 단 1회 수행되는 경우가 더 많기 때문이다. 만약 2차 이상의 미분이 필요하다면 `create_graph=True` 로 설정하면 된다.


### 32.4 __init__.py 변경 
지금까지의 수정 내용을 바탕으로 `dezero/core.py` 에 반영한다. 이제는 `dezero/core_simple.py` 대신 `dezero/core.py`를 사용할 것이므로 `dezero/__init__.py` 를 수정한다. 

```python
# =============================================================================
# step23.py부터 step32.py까지는 simple_core를 이용
is_simple_core = False
# step33 부터는  dezero/core.py 로 대체한다.

# =============================================================================

if is_simple_core:
    from dezero.core_simple import Variable
    from dezero.core_simple import Function
    from dezero.core_simple import using_config
    from dezero.core_simple import no_grad
    from dezero.core_simple import as_array
    from dezero.core_simple import as_variable
    from dezero.core_simple import setup_variable
    from dezero.core_simple import Config

else:
    # step33 부터 dezero/core.py 정의
    from dezero.core import Variable
    from dezero.core import Function
    from dezero.core import using_config
    from dezero.core import no_grad
    from dezero.core import as_array
    from dezero.core import as_variable
    from dezero.core import setup_variable
    from dezero.core import Config
```